In [86]:
import os
import gpxpy
import gpxpy.gpx
import numpy as np
import json
from glob import glob

In [87]:
def parse_data():
    gpx_files = []
    vectors = []
    
    for file in glob("gpx_data/*.gpx"):
        gpx_file = gpxpy.parse(open(file))
        gpx_files.append(os.path.basename(file))
        
        for track in gpx_file.tracks:
            vector = [(point.latitude, point.longitude) for segment in track.segments for point in segment.points]
            vectors.append(vector)
    return vectors, gpx_files

In [88]:
def parse_request_data(json_data):
    paths = []
    for json_object in json.loads(json_data):
        users.append(json_object['user'])
        
        with open("gpx_data/{}.gpx".format(json_object['user']), 'w') as f:
            f.write(json_object['gpx'])
            
        gpx_file = gpxpy.parse(json_object['gpx'])
        
        for track in gpx_file.tracks:
            path = [(point.latitude, point.longitude) for segment in track.segments for point in segment.points]
            paths.append(path)
            
        break # request should only have 1 person
    
    return paths, json_object['user']

In [82]:
# with open("gpx_data/20150909_185946.gpx") as f:
#     j = [{
#         'user' : '0',
#         'gpx' : f.read()
#     }]
    
#     j = json.dumps(j)
    
# paths, users = parse_data(j)

In [83]:
class BoundingBox(object):
    """
    A 2D bounding box
    """
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
        self.midx = (self.minx + self.maxx) / 2
        self.midy = (self.miny + self.maxy) / 2
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny

    def mid(self):
        return [self.midx, self.midy]
    def __repr__(self):
        return "BoundingBox({}, {}, {}, {})".format(
            self.minx, self.miny, self.maxx,self.maxy)

In [84]:
def score():
    score_matrix = np.array([np.array([np.inf for _ in range(len(paths))], dtype="float64") for _ in range(len(paths))])
    for i in range (len(paths)):
        for j in range (i+1, len(paths)):
                b1 = BoundingBox(paths[i])
                b2 = BoundingBox(paths[j])
                m1 = b1.mid()
                m2 = b2.mid()
                score = ((m2[0] - m1[0]) **2 + (m2[1] - m1[0]) **2) ** 1/2
                score_matrix[i][j] = score
                score_matrix[j][i] = score

    score_matrix=np.array(score_matrix).reshape(len(paths), len(paths))
    score_matrix            

In [89]:
# # Top recommendations for User 0
# tops = score_matrix[0].argsort()[:5]
# results = []
# for top in tops:
#     results.append(suggestions[top]) 
    
# results

In [94]:
# Main entry point
def handle_request(json_data):
    paths, gpx_files = parse_data()
    user_path, user = parse_request_data(json_data)
